# Predicting Gene Ontology (GO) Aspects with Fine-Tuned Text Transformer

High-throughput [DNA sequencing](https://en.wikipedia.org/wiki/DNA_sequencing#High-throughput_sequencing_(HTS)_methods) has give us the power to sequence the entire genome of a species within a day. With the sequence on hands, we can deploy **computational models to identify and annotate genes** based on their characteristics, attributing correct functions and subcellular locations of the gene product given the gene sequence. For example, the [Critical Assessment of Protein Function Annotation (CAFA)](https://www.cell.com/trends/genetics/fulltext/S0168-9525(13)00166-2) competition engages the data science community in enhancing protein prediction by utilizing features derived from [Gene Ontology (GO)](https://geneontology.org/docs/ontology-documentation/).

The GO is represented in directed knowledge graph where **each node represents a specific GO term**, defining a particular aspect of genes and their products. These GO terms are categorized into three major aspects:

- <font color="grey">***Molecular Function (MF)***</font>: These terms define the **activities** performed by gene products, such as *catalysis* or *transport*. These functions can be further refined by more specific GO terms, for example, "protein kinase activity" within the broader category of "catalysis".

- <font color="grey">***Cellular Component (CC)***</font>: These terms specify the subcellular **locations** of gene products, including compartments like *chloroplast* or *nucleus*, as well as macromolecular complexes like *proteasome* or *ribosome*.

- <font color="grey">***Biological Process (BP)***</font>: These terms delineate the biological **pathways** in which gene products are involved, ranging from 'DNA repair' and 'carbohydrate metabolic process' to overarching processes like *biosynthetic processes*.

The terms are **hierarchically organized nodes with edges connecting terms based on their relationships**. For more information about how the GO graph is structured, please refer to my previous post [GO-graph-EDA](https://github.com/matiollipt/GO-graph-EDA) and the [Gene Ontology reference](https://geneontology.org/docs/ontology-documentation). For now, it is essential to know that **each node representing a GO term has specific attributes**.

A feature often overlooked when deploying the GO graph to assist in the prediction and classification of gene functions is the **textual definition of each GO term**. For example, the GO term ID [GO:0015986](https://www.ebi.ac.uk/QuickGO/term/GO:0015986) is defined as *"The transport of protons across the plasma membrane to generate an electrochemical gradient (proton-motive force) that powers ATP synthesis."*, along with other attributes shown below:

---
 **GO id**: GO:0015986

- **name**: 'proton motive force-driven plasma membrane ATP synthesis'
- **namespace**: 'biological_process'
- **def**: '"The transport of protons across the plasma membrane to generate an electrochemical gradient (proton-motive force) that powers ATP synthesis." [GOC:mtg_sensu, ISBN:0716731363]'
- **synonym**: ['"ATP synthesis coupled proton transport" BROAD []', '"plasma membrane ATP synthesis coupled proton transport" EXACT []']
 - **is_a**: ['GO:0015986']}

 ---

In this first simple exercise, we will use the text definitions of GO terms to **fine-tune** a pre-trained Large Language Model (LLMs) based on the [transformer architecture](https://en.wikipedia.org/wiki/Transformer_(machine_learning_model)). The **goal** is to leverage this LLM to **categorize GO term definitions** based on their alignment with the major GO ontologies (BP, CC, and MF).

The transformer architecture, proposed by the [Google Brain Team](https://research.google/) in the seminal paper ["Attention Is All You Need"](https://arxiv.org/abs/1706.03762), revolutionized [Natural Language Processing (NLP)](https://en.wikipedia.org/wiki/Natural_language_processing) by introducing the idea of using solely **attention mechanisms** to identify dependencies between words in a text sequence. This approach contrasts with previous ones by dropping **recurrent layers** in [Long Short-Term Memory (LSTM)](https://en.wikipedia.org/wiki/Long_short-term_memory) networks to handle (i.e., keep track of previous words/tokens) text sequences. In the Transformer architecture, the order of the sequence is retained through **positional encoding**, and the **dependencies** are computed using multi-head attention to keep track of more than one dependency in the input sequence.


**Transformers** have gained significant attention in the machine learning landscape, contributing to the development of generative models like [chatGPT](https://chat.openai.com). Text transformer-based architectures are specifically engineered to analyze the relationships between words in sentences, considering their positions to accuratelly capture the meaning of the text. This capability is instrumental in tasks such as text classification and ranking, as well as enabling generative models to generate new sentences in response to queries.

>**Fine-tuning** is a powerful technique in machine learning that involves using models that have undergone **semi-supervised training** on extensive data from various sources. This process imparts the model with underlying semantic features of the language. After pre-training, the model is further trained in a **supervised manner** for a specific task, such as text generation, classification, or label prediction (for instance, identifying whether emails are spam or not). During this phase, the deeper encoded features of the pre-trained model are retained, but the trainable parameters of the output layer(s) are adjusted. For example, a feed-forward neural network can be added for binary classification of "spam" vs "not spam." These steps embody the concept of [Transfer Learning](https://machinelearningmastery.com/transfer-learning-for-deep-learning/), a powerful machine learning technique that involves applying knowledge learned from one task effectively to another task.

In the **next post**, we will fine-tune the BERT model to **generate better embeddings of GO term definitions for protein function prediction**.

In [ ]:
# install required libraries (uncomment these lines for Colab or Kaggle)

# !pip install obonet
# !pip install nltk
# !pip install transformers
# !pip install datasets
# !pip install wordcloud
# !pip install bertviz


In [ ]:
# imports

from pathlib import Path
import re
import random
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

# set home directory

home_dir = Path("../GO-graph-definition-text-transformer/")
print(f"Home directory: {home_dir.as_posix()}")

In [ ]:
# convenience functions for this notebook
import matplotlib.pyplot as plt
from pylab import rcParams
import seaborn as sns
import networkx as nx
from sklearn.metrics import confusion_matrix, balanced_accuracy_score

# plot confusion matrix
def plot_cm(y_true: pd.Series, y_pred: pd.Series):
    # print out the balanced accuracy score
    print(
        f"Balanced Accuracy Score: {balanced_accuracy_score(y_pred, y_true):.1%}")

    # create normalized confusion matrix
    cm = confusion_matrix(
        y_true,
        y_pred,
        normalize="pred",
    )

    # convert to dataframe for easy plotting
    cm_df = pd.DataFrame(
        cm,
        index=["Biological Process", "Cellular Component", "Molecular Function"],
        columns=["Biological Process",
                 "Cellular Component", "Molecular Function"],
    )

    plt.figure(figsize=(10, 8))
    sns.set(font_scale=1.2)
    sns.heatmap(
        data=cm_df,
        annot=True,
        cmap="Blues",
    )
    plt.title("Confusion Matrix")
    plt.ylabel("True")
    plt.xlabel("Predicted")
    plt.show()


# plot graphs or sub-graphs
def plot_graph(
    G: nx.Graph(),
    plot_nodes=list,
    plot_all=False,
    num_nodes=100,
    iterations=100,
    figure_size=(8, 8),
    node_color="green",
    font_color="black",
):
    """
    Plot the full graph (plot_all=True) or a sub-graph with the
    nodes passed as parameter. The 'n_terms' parameter controls
    how many nodes will be plotted in the top-down list of nodes.

    G: nx.Graph
    plot_nodes: list with the nodes to plot
    plot_all: select wheter plot the full graph or a subgraph
    n_nodes: number of nodes to plot
    iterations: number of iterations of spring_layout algorithm
    figure_size: tuple with figure dimensions (W x H)
    node_color: specify node color
    """
    if plot_all:
        n_nodes = len(G.nodes)
        n_edges = len(G.edges)

    else:
        G = nx.subgraph(G, plot_nodes[:num_nodes])
        n_nodes = len(G.nodes)
        n_edges = len(G.edges)

    # set figure size
    rcParams["figure.figsize"] = figure_size

    # nodes features
    degree_dict = dict(G.degree)  # get degree to adjust node viz
    node_size = list([x * 100 for x in degree_dict.values()])
    node_alpha = [float(x / max(degree_dict.values()))
                  for x in degree_dict.values()]

    pos = nx.spring_layout(
        G,
        scale=10,
        k=25 / np.sqrt(n_nodes),
        seed=13,
        iterations=iterations,
    )

    ax = nx.draw_networkx_nodes(
        G, pos, node_size=node_size, alpha=node_alpha, node_color=node_color
    )

    ax = nx.draw_networkx_edges(
        G, pos, width=0.5, node_size=node_size, edge_color="grey"
    )

    ax = nx.draw_networkx_labels(
        G,
        pos,
        font_size=10,
        font_color=font_color,
        horizontalalignment="center",
        verticalalignment="center",
    )

    plt.title(f"Graph plot with {G.order()} nodes and {G.size()} edges")
    plt.axis("off")
    plt.show()


## Dataset Preparation

Our initial step involves pre-processing the dataset to make it compatible with the pre-trained model that we are going to fine-tune according to our needs. One model that fits our task requirements is the [bert-base-multilingual-uncased](https://huggingface.co/bert-base-multilingual-uncased), based on the well-known [Bidirectional Encoder Representations from Transformers (BERT)](https://huggingface.co/docs/transformers/model_doc/bert) architecture. This model has previously undergone **semi-supervised training** on an extensive dataset comprising millions of entries from [Wikipedia](https://www.wikipedia.org/) in 102 different languages. During this pre-training phase, the model learns language rules and dependencies, setting the stage for its application in various **supervised training** scenarios. These applications include tasks like sentiment analysis, text generation, text sequence classification, and, in our specific case, identifying the major GO ontology categories (BP, CC, and MF) for the given text sequences.

To prepare the dataset for fine-tuning our model, we will perform the following tasks:

1. **Feature Extraction**: In this step, we convert the attributes associated with each GO term's nodes into a Pandas DataFrame. This conversion streamlines the creation of pre-processed and tokenized datasets that will be used to train the model. We will focus on extracting only the text definitions and labels corresponding to the aspects we aim to predict (BP, MF, and CC).

2. **Dataset Creation and Splitting**: Our data will be divided into two subsets: a training set (80%) and an test set (20%). These sets will contain the input text derived from the definitions of GO terms.

3. **Text Tokenization**: The input text undergoes a tokenization process, breaking it into smaller units known as tokens. Special tokens are also incorporated to indicate the start and end of sequences and sentences. This step is crucial for enabling the model to comprehend the text.

To create and manage these datasets, we will make use of the [Dataset](https://huggingface.co/docs/datasets/index) library provided by Hugging Face. Additionally, as our project progresses, we will leverage pre-trained sequence classification models and their corresponding tokenizers, which are also available from [Hugging Face](https://huggingface.co/).

### Feature Extraction

The GO graph is stored in the *OBO (Open Biomedical Ontologies) file format*, designed specifically for the construction and representation of biological ontologies. To convert this file format into a **NetworkX** object, we can utilize the Python library [obonet](https://github.com/dhimmel/obonet). [NetworkX](https://networkx.org/) offers a robust framework for graph manipulation and analysis. Below, we load the GO graph and visualize nodes with high degrees using our convenience function `plot_graph()`:

In [ ]:
from obonet import read_obo

# read and parse GO graph data
go_graph = read_obo(home_dir.joinpath("data/go-basic.obo"))
print(f"The GO graph is a {go_graph}.")

# select nodes with high degree: the list comprehension yields
# GO graph nodes with degree higher than 20, # i.e., only nodes
# with more than 20 connected neighbors, for plotting.
node_list = [n for n in go_graph.nodes() if go_graph.degree(n) > 20]

# plot subgraph
plot_graph(
    G=go_graph,
    plot_nodes=node_list,
    plot_all=False,
    num_nodes=300,
    iterations=500,
    font_color="grey",
)

Recall from above that the nodes encompass attributes describing the corresponding GO terms. By running the cell above a couple of times, you start to notice that all GO terms contain the following essential attributes:

>***name***: unique identifier of the term in a human-readable format

>***namespace***: one of the three major ontologies (MF, CC or BP) to which the term belongs

>***definition***: a short description of what the GO term means for humans. It can also contains references to publications defining the term (e.g. PMID:10873824).

There are additional attributes of each node corresponding to GO terms, but we won't be using them in this project.

In [ ]:
go_graph.nodes()[random.choice([x for x in go_graph.nodes()])]


Our upcoming task involves **extracting attributes from GO graph nodes and structuring them into a dataframe**. This process aims to facilitate the handling of text definitions. Additionally, we will include a column indicating the word count for each definition and visualize the distribution of these counts. This distribution will provide valuable insights for choosing optimal hyperparameters for tokenization (i.e., breaking the text into individual words or tokens) and fine-tuning our selected pre-trained model.

Looking at the sample GO term above, the *definition* ('*def*') is not purely textual. Here is the definition of the GO term **GO:0150058** as another example:

<font color="grey">'def': '<u>"A G protein-coupled receptor complex that serves as a receptor for amylin polypeptide (AMY) and consists of a calcitonin receptor (CTR/CALCR) and a receptor activity-modifying protein (RAMP) 3. Amylin receptor complex 3 (AMY3) also serves as a receptor for the amyloid-beta complex. Ligand binding to AMY3 results in increased cytosolic calcium ion levels and in activation on multiple intracellular signalling pathways."</u> [GOC:aruk, GOC:bc, PMID:22500019]'</font>

We intend to extract only the textual description from the definition to input the model during the fine-tuning step. Therefore, we need to split the text using appropriate separators and employ indexes to access the desired information. Let's examine the content of a single node to understand how to extract its contents. Please note that the nodes' attributes are stored in a tuple where the first element is the **GO term ID**, and the second is a **dictionary containing the respective attributes**.

In [ ]:
for item in go_graph.nodes.items():
    print(f"type: {type(item)}")
    print(f"item: {item}")
    print(f"GO term id: {item[0]}")
    print(f"attributes: {item[1]}")
    break


In [ ]:
# create GO definitions dataframe

# create empty dataframe to store nodes' attributes
go_df = pd.DataFrame(
    columns=["go_id", "name", "aspect", "definition", "def_word_count"]
)

# iterate over nodes to extract dictionary keys and values
n_rows = len(go_graph.nodes)

for idx, item in tqdm(enumerate(go_graph.nodes.items()), total=n_rows):
    go_term = item[0]
    name = item[1]["name"]
    aspect = item[1]["namespace"]

    # split 'def' content and get the text definition only
    definition = item[1]["def"].split(sep='"', maxsplit=2)[1]

    # count the number of words of the text we just extracted
    def_word_count = len(re.findall(r"\w+", definition))

    go_df.loc[idx] = [
        go_term,
        name,
        aspect,
        definition,
        def_word_count,
    ]

# save dataframe
home_dir.joinpath("data/").mkdir(parents=True, exist_ok=True)
go_df.to_csv(home_dir.joinpath("data/go_df.csv"), index=False)

display(go_df.head())

# word count boxplot
plt.figure(figsize=(2, 4))
plt.title("Dataset Word Count")
plt.boxplot(go_df.def_word_count.values)
plt.ylabel("Word Count")
plt.show()

Before proceeding, we will make some modifications in our dataset: renaming the **definition** column to *text*, converting the **aspect** column to a **categorical data type** while mapping the aspects to numeric labels and keeping only these two columns for the downstream tasks.

In [ ]:
# loading saved dataframe with GO graph nodes' attributes

# load saved GO dataframe
go_df = pd.read_csv(home_dir.joinpath("data/go_df.csv"))

# convert categorical labels to numbers (aspect)
go_df["aspect"] = pd.Categorical(go_df["aspect"])

# get categorical codes
go_df["label"] = go_df["aspect"].cat.codes

# select only relevant columns
data = go_df[["definition", "label"]].copy()

# rename definition column
data.rename(columns={"definition": "text"}, inplace=True)

print(f"Dataset dimensions: {data.shape}")
display(data.head())

# print label mapping for reference
print("\nCode: label")
for code, aspect in enumerate(go_df.aspect.cat.categories):
    print(f"{code}: {aspect}")


We can visualize the most common terms in the dataset by creating a [word cloud](http://amueller.github.io/word_cloud/). This visual representation is quite helpful in identifying the most frequently occurring words in a given text.

Below, we can clearly see that life is all about a system of **controled catalysis**, and the information encoded in the genomes allows such organized system to be passed forward.

In [ ]:
from wordcloud import WordCloud

go_wordcloud = WordCloud(
    width=800,
    height=800,
    background_color="white",
    min_font_size=10,
    colormap="rainbow",
    random_state=13,
).generate(data.text.to_string())

plt.figure(figsize=(8, 8), facecolor=None)
plt.suptitle("Word cloud of GO term definitions")
plt.imshow(go_wordcloud)
plt.axis("off")
plt.tight_layout(pad=1)
plt.show()


### Dataset Creation and Splitting

To construct the dataset containing our text sequences and corresponding labels for fine-tuning the model, we will leverage the [Dataset](https://huggingface.co/docs/datasets/package_reference/main_classes#datasets.Dataset) class provided by Hugging Face. Using its built-in dataset split method, we will create distinct training and testing sets. We will ensure that the sampling is **stratified by label**. This stratification is crucial as it **preserves the distribution of the GO ontologies**, ensuring a representative balance in our datasets.

In [ ]:
from datasets import Dataset

# create dataset
dataset = Dataset.from_pandas(data)

# change label column to ClassLabe to allow stratification
dataset = dataset.class_encode_column("label")

# split the dataset into train (80%) / test (20%)
dataset = dataset.train_test_split(test_size=0.2, stratify_by_column="label")
dataset


### Text Tokenization

The **tokenization strategy must align with the chosen model**. This is crucial to ensure that tokens are mapped to the **same indices** presented to the model during training and that the **same special tokens** are used to denote the beginning of a text sequence and the separation between sentences. We also need to handle inputs that are longer or shorter than the input accepted by the model, or defined by us when we initialize the tokenizer.

> **Handling text input length with truncation and padding strategy**: The model accepts fixed-sized tensors for training and evaluation. To ensure that we feed the model with fixed-sized tensors, the tokenizer handle sequences of varying lengths by truncating and padding longer and shorter text sequences, respectively. If the *truncation* parameter is set to 'True,' input sequences longer than a defined length will be truncated. Conversely, if the sequences are shorter, the special padding token [PAD] will be added until the number of tokens in the input matches our requirements for the model of choice. Truncation and padding are pivotal in guaranteeing consistent input sizes, as the inputs for the model we chose are predefined fixed-size tensors (e.g., 510 tokens + 2 special tokens [CLS] and [SEP] = 512). For our **fine-tunning** task, setting the input maximum length to 150 tokens encompass the majority of the samples and expedite the tokenization process.

**Classification Token [CLS]** and **Separator Token [SEP]** are special tokens that provide information about the input provided to the model we choose. [CLS] marks the initiation of the sequence for the BERT model. [SEP] separates sentences in sentence-pair tasks. It aids the model in capturing the relationships between two sentences that are concatenated using [SEP]. Note the use of padding [PAD] to fill-up the input sequence to a fixed length. 

In this context, we are employing the [BertTokenizerFast](https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertTokenizerFast) with the tokenization strategy of the pre-trained model [bert-base-multilingual-uncased](https://huggingface.co/bert-base-multilingual-uncased) to convert text to lowercase and eliminate capitalization. Lowercase inputs typically result in a smaller number of tokens and tend to generalize better for unseen text sequences during production phase. However, we can later explore the possibility of using cased inputs to evaluate model performance.

As indicated by the boxplot above, all GO term definitions conform to the input size requirements for fine-tuning (510 tokens). Nonetheless, we might consider limiting the input length to expedite the fine-tuning process. To proceed, we will create the dataset with training and test sets using Hugging Face's Dataset library. We will employ stratification to maintain the distribution of labels between datasets.

In [ ]:
from transformers import BertTokenizerFast

# initiate tokenizer with parameters from the pre-trained model
tokenizer = BertTokenizerFast.from_pretrained("bert-base-multilingual-uncased")


# define tokenize function to be applied to the input (lowercase is applied by default)
def tokenize_function(examples):
    return tokenizer(examples["text"], padding=True, max_length=100, truncation=True)


# tokenize dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset


In [ ]:
print("Dataset features:")
print("text:", tokenized_dataset["train"]["text"][0])
print("label:", tokenized_dataset["train"]["label"][0])
print("input_ids:", tokenized_dataset["train"]["input_ids"][0])
print("token_type_ids:", tokenized_dataset["train"]["token_type_ids"][0])
print("attention_mask:", tokenized_dataset["train"]["attention_mask"][0])


Each feature holds specific information relevant to the tokenized dataset:

- **text**: the input text sequence.
- **label**: numerical values corresponding to the classes (i.e. ontologies) to predict:
  - 0: biological_process
  - 1: cellular_component
  - 2: molecular_function
- **input_ids**: These are the indices of each token within the sequence. These indices were generated during the model's pre-training phase and are now applied to our input.
- **token_type_ids**: This feature provides references to the sentence to which each token belongs.
- **attention_mask**: It indicates whether the token should be attended to during processing.

We can decode the the **input_ids** to obtain the original text. Below we print out the decoded definitions based on the input_ids. In the output we can see the **special tokens** we've been discussing and the results of our **truncation and padding** strategy. I did some changes in the print for better visualization of the decoded input ids and the input ids.

In [ ]:
dataset_len = len(tokenized_dataset["train"]["input_ids"])
random_indices = random.sample(range(0, dataset_len), 5)

for idx in random_indices:
    print(tokenizer.decode(tokenized_dataset["train"]["input_ids"][idx]))
    input_ids_list = tokenized_dataset["train"]["input_ids"][idx]
    print(input_ids_list)
    print(f"Input length: {len(input_ids_list)}")
    print(50 * "=")


## Fine-Tuning the Model

Continuing our progress, we will initialize the [BertForSequenceClassification](https://huggingface.co/docs/transformers/v4.33.2/en/model_doc/bert#transformers.BertForSequenceClassification) model and load the **weights** from the [pre-trained model](https://huggingface.co/bert-base-multilingual-uncased). Additionally, we will specify the number of target labels we aim to predict. In our case, we intend to classify whether the text corresponds to Biological Process (label = 0), Cellular Component (label = 1), or Molecular Function (label = 2). Consequently, the number of labels is set to **3**.

It's crucial to recognize that training Large Language Models can be computationally intensive, often demanding substantial hardware resources. The model we are utilizing consists of **168 million trainable parameters**, pre-trained in a semi-supervised manner. While we won't be training all of these parameters, but only a subset for our specific task, it remains a significant computational task.

>The **carbon footprint** of such large language models can be quite taxing. An additional important benefit of fine-tunning is reducing the carbon footprint when deploying custom models. This [interesting seminar](https://www.youtube.com/watch?v=_6R7Ym6Vy_I) from [The Royal Institution](https://youtube.com/@TheRoyalInstitution?si=_eU1H86ZJxwWbyqj) YouTube channel talks about it.

Depending solely on a CPU for training and fine-tuning may lead to lengthy and impractical processing times. Therefore, it is essential to assess the availability of GPU resources, verify the allocated memory, and ensure we have ample space for efficient data processing.

Your sentence is grammatically correct and clear. There is only one minor suggestion to enhance readability:

>Hugging Face's [Trainer](https://huggingface.co/docs/transformers/v4.29.0/en/main_classes/trainer#transformers.Trainer) seamlessly handles model and data allocation between devices. If a GPU is available and correctly configured in the system, the Trainer class will utilize it.

In [ ]:
import torch

print(f"cuda available?: {torch.cuda.is_available()}")
print(torch.cuda.memory_summary())

# free-up memory
torch.cuda.empty_cache()


In [ ]:
from transformers import BertForSequenceClassification

# initialize 'bert-base-multilingual-uncased pre-trained model
model = BertForSequenceClassification.from_pretrained(
    "bert-base-multilingual-uncased", num_labels=3
)

model


### Fine-Tuning with Trainer

Our next step involves fine-tuning the model using the [Trainer](https://huggingface.co/docs/transformers/v4.33.2/en/main_classes/trainer#transformers.Trainer) class, a specialized tool designed to optimize the training of Hugging Face's models. Subsequently, we will explore fine-tuning the model using native PyTorch methods.

To configure the Trainer effectively, it's essential to set the **hyperparameters** utilizing the [TrainingArguments](https://huggingface.co/docs/transformers/v4.33.2/en/main_classes/trainer#transformers.TrainingArguments) and specify the evaluation metrics through the [evaluate](https://huggingface.co/docs/evaluate/index) module. This strategic setup allows us to closely monitor the training progress and evaluate the model's performance after the fine-tuning process.

In [ ]:
# PyTorch Trainer
from transformers import TrainingArguments

train_args = TrainingArguments(output_dir="test_trainer")

# list training parameters
args_list = [
    x for x in list(dir(train_args)) if not re.compile(r"\b_[A-Za-z0-9_]*\b").match(x)
]
print(f"There are {len(args_list)} training arguments: {args_list}")


#### Metrics

Given that the Trainer lacks an automatic performance evaluator during training, it's crucial to integrate metrics and pass them to the Trainer object through TrainingArguments.

For our performance assessment, we will employ the ROC/AUC score. To facilitate this, we will create a function to convert **predictions** (probabilities ranging from 0 to 1, inclusive, obtained from the softmax function) into **logits** (the raw output from the model with unnormalized scores). This transformation aligns with the explanation provided in [this Hugging Face tutorial](https://huggingface.co/docs/transformers/training). By incorporating this custom metric, we can effectively evaluate the model's performance during the fine-tuning process.

In [ ]:
import evaluate

# list metrics
metrics_list = evaluate.list_evaluation_modules()
print(f"There are {len(metrics_list)} available metrics: {metrics_list}")

# load roc_auc metric
metric = evaluate.load("accuracy")


# convert preds --> logits
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


# set evaluation for training
train_args = TrainingArguments(
    output_dir="test_trainer",
    evaluation_strategy="epoch",
)


To reduce the training duration and make it more suitable for a tutorial, we will fine-tune the model using a smaller subset of the training data, reserving 20% of it for validation purposes.

In [ ]:
small_train_dataset = tokenized_dataset["train"].shuffle(
    seed=42).select(range(2000))
small_eval_dataset = tokenized_dataset["test"].shuffle(
    seed=42).select(range(2000))

print(small_train_dataset)
print(small_eval_dataset)


Finally, we can initiate the model training process. The Trainer class, provided by Hugging Face, simplifies the training procedure to just a few lines of code. Given all the information we've covered so far, the parameters to be passed to the Trainer become quite self-explanatory.

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()


## Fine-Tuning with Native PyTorch

Having explored the conveniences of Hugging Face's classes and models, it's valuable to delve into the internal workings of the training loop. Understanding these mechanisms equips you to make refinements that could enhance the model's performance or render it more lightweight for deployment.

In this section, we will fine-tune our BERT model using native PyTorch. This approach entails implementing the training loop and creating the dataloader responsible for supplying batches of examples during training. By proceeding with the exercise of fine-tunning the model using native PyTorch, we gain a deeper understanding of the training process, fostering the ability to make tailored adjustments to achieve the desired model's performance in our classification task.

But first we might want clean-up GPU's memory and delete the *model* and *trainer* objects to  release space and set the model to the initial state. Optionally, we can restart the notebook.

In [ ]:
del model
del trainer
torch.cuda.empty_cache()


### Dataloader

To facilitate the training process, we need to configure the PyTorch [DataLoader](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader), which combines the dataset structure and a sampler to provide batched chunks of training data to the model during training. The DataLoader is optimized for memory occupancy and speed, making it ideal for our needs. However, before we proceed, we must make some modifications to the tokenized dataset to align it with the input requirements of the model:

1. Remove the *text* column from the dataset since the model does not accept text as input.
2. Rename the *label* column to *labels* as the model is preset with this specific column name.
3. Instruct the dataset to return PyTorch tensors.
4. Select only a small portion of the dataset for fine-tuning the model, similar to what we did before.
5. Instantiate the data loaders for training and test sets, shuffling the training samples but not the evaluation set. Also, batch sizes tends to be small during training and fine-tunning of such large models due to memory constraints.

In [ ]:
from torch.utils.data import DataLoader

# remove the text column
tokenized_dataset = tokenized_dataset.remove_columns(["text"])

# rename the label column
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")

# set torch tensors as the dataset output
tokenized_dataset.set_format("torch")

# create smaller training and test subsets
small_train_dataset = tokenized_dataset["train"].shuffle(
    seed=42).select(range(2000))
small_eval_dataset = tokenized_dataset["test"].shuffle(
    seed=42).select(range(2000))

# configure torch data loader (eval set don't should be shuffled)
train_loader = DataLoader(small_train_dataset, shuffle=True, batch_size=8)
eval_loader = DataLoader(small_eval_dataset, batch_size=8)

print("Train:\n", small_train_dataset)
print("\nTest:\n", small_eval_dataset)


### Optimizer and Learning Rate

The [optimizer](https://pytorch.org/docs/stable/optim.html) adjusts the weights of the network during training to minimize the **loss** computed by a [loss function](https://pytorch.org/docs/stable/nn.html#loss-functions), i.e., minimize the discrepancy between the input and output of the model. Some optimization algorithms, such as [RMSProp](https://pytorch.org/docs/stable/generated/torch.optim.RMSprop.html#torch.optim.RMSprop), [Adam](https://pytorch.org/docs/stable/generated/torch.optim.Adam.html#torch.optim.Adam) and [AdamW](https://pytorch.org/docs/stable/generated/torch.optim.AdamW.html#torch.optim.AdamW), dynamically adapts the [learning rate]() of each network parameter based on the gradient descent previous magnitudes.

The recommended optimizer to fine-tune BERT models is AdamW, a variant of Adam that implements weight decay to improve model's convergence and generalization power by preventing [overfitting](https://en.wikipedia.org/wiki/Overfitting). We will implement an [scheduler](https://huggingface.co/docs/transformers/main_classes/optimizer_schedules#transformers.get_scheduler) to adapt the learning rate along the training process.

>Remember that **overfitting** happens when the model achieves high scores in predicting the training data output but has **little generalization power for unseen data**.

First things first, let's instantiate the model with pre-trained weights and move it to the GPU memory:

In [ ]:
import torch
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(
    "bert-base-multilingual-uncased", num_labels=3
)

# check GPU availability
device = torch.device(
    "cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

model


In [ ]:
from torch.optim import AdamW
from transformers import get_scheduler

# implement optimizer with small learning rate (recommended for BERT)
optimizer = AdamW(model.parameters(), lr=5e-5)

# implement scheduler
num_epochs = 3
num_training_steps = num_epochs * len(train_loader)  # total number of batches

lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

print(f"Number of training steps: {num_training_steps} (# epochs x # batches)")
optimizer


### Training Loop

Now we can train, I mean, fine-tune the model. The training loop controls the training process by allocating the batches of data to the model's device location and trigger the computation of the loss and optimization steps. In the training loop we also specify the number of epochs that we will be training our model.

To monitor the training progress, we can set a progress bar using the handy [tqdm](https://github.com/tqdm/tqdm) library. We can just wrap any interable with tqdm that it will take care of the rest.

In [ ]:
# the total number of batches going through the model
progress_bar = tqdm(range(num_training_steps))

# activate the training mode
model.train()

# the loop itself
for epoch in range(num_epochs):
    for batch in train_loader:
        # moves the data to the specified device (8 samples per batch)
        batch = {key: value.to(device) for key, value in batch.items()}
        # unpack batch dict and get the outputs to compute loss
        outputs = model(**batch)
        # extract the loss attribute from the output object
        loss = outputs.loss
        # compute gradients (derivatives)
        loss.backward()
        # update the weights using the computed gradients
        optimizer.step()
        # adjust the learning rate after each pass
        lr_scheduler.step()
        # reset the gradients because the weights are updated
        # in each pass using only the gradients of the batch.
        optimizer.zero_grad()
        progress_bar.update(1)

### Evaluation

To assess the performance of the trained model, it's essential to define the metric (just as we did when fine-tuning the model using the Trainer class) and create the evaluation loop. During evaluation, we provide batches to the model, accumulate predictions in the defined metric's attributes, and calculate the final score.

Before evaluation, we switch the model to evaluation mode by invoking the model's `eval()` method. This action alters the behavior of certain layers that behave differently between training and evaluation, such as Dropout and Batch Normalization layers. It also informs the model not to compute gradients. Here's why:

1. **Dropout**: Dropout randomly deactivates a fraction of neurons during training to prevent overfitting. In evaluation mode, this operation is unnecessary because we aim to predict all samples consistently.

2. **Batch Normalization**: During training, each batch is normalized independently. In evaluation mode, we accumulate batches and perform normalization at the end considering the population statistics (mean and variance) learned during training.

3. **Gradient Computation**: Since no weights are updated during evaluation mode, disabling gradient computation saves memory and processing time. We also use the PyTorch context manager `torch.no_grad()` to disable gradient computation in the evaluation loop.

Setting the model to evaluation mode ensures that it behaves consistently and produces reliable predictions during the evaluation process.


In [ ]:
import evaluate

# define metric
metric = evaluate.load("accuracy")

# set the model to evaluation mode
model.eval()

# evaluation loop
for batch in eval_loader:
    batch = {key: value.to(device) for key, value in batch.items()}

    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

print(f"Accuracy: {metric.compute()['accuracy']}")


## Using the Fine-Tunned Model for Predictions

We've fine-tunned our model to understand the specific semantics of GO terms definitions regarding to which aspect the sentence is more likely to belong: Biological Process (BP), Cellular Component (CC) and Molecular Function (MF). An interesting possible application is using the model to classify any sentence regarding these aspects. One could ask: What is the predominantly GO aspect in a scientific text or sentence?

We could determine the predominant GO aspect (biological process, celullar localization or molecular function) of single sentence and a paper abstract on molecular biology.

Let's start by examining a single sentence sample:

In [ ]:
# This is just a sample; the predictions make sense within the context they were fine-tuned for

# just a matter of curiosity, this is the NASA's definition of life when searching for life elsewhere,
# and we could say that is the shortest and concise definition you may find suitable.

sample = "Life is a self-sustaining chemical system capable of Darwinian evolution."

As we did before, we tokenize the text sequence using the same parameters that we've used to configure the tokenizer for fine-tunning the model:

In [ ]:
# tokenize input
inputs = tokenizer(
    sample, max_length=100, truncation=True, padding=True, return_tensors="pt"
).to(device)

# switch model to evaluation mode
model.eval()

# get logits
with torch.no_grad():
    logits = model(**inputs).logits

# calculate probabilities from logits
probs = torch.nn.functional.softmax(logits, dim=1)
prediction = torch.argmax(probs, dim=1).item()

# print label probabiliteis
print("Probabilities:")
for code, aspect in enumerate(go_df.aspect.cat.categories):
    print(f"{aspect}: {probs[0][code]:.3f}")

print(f"\nPredicted Ontology: {go_df.aspect.cat.categories[prediction]}")

In [ ]:
# download / update punkt, the nltk's recommended sentence tokenizer

import nltk

nltk.download("punkt")

In [ ]:
# sample_text = "sentences here"

# load sample text
file = open(home_dir.joinpath("data/sample_text.txt"), "r").read()

# split sample text into sentences and put into a dataframe
sentences_list = nltk.tokenize.sent_tokenize(file)
sentences_df = pd.DataFrame(columns=["sentence"], data=sentences_list)
print(f"Number of sentences: {len(sentences_list)}")

# switch to evaluation mode
model.eval()

# empty list to store temporary dictionaries with samples' predictions
data_list = []

# iterates over the sentences' list, tokenize, predict and append results
for sample in tqdm(sentences_list, total=len(sentences_list)):
    inputs = tokenizer(
        sample,
        max_length=100,
        truncation=True,
        padding=True,
        return_tensors="pt",
    ).to(device)

    with torch.no_grad():
        logits = model(**inputs)
        prediction = torch.nn.functional.softmax(logits.logits, dim=1)

    # get predictions out of GPU's memory, convert to list for appending
    prediction_list = prediction.cpu().numpy().tolist()[0]

    data_dict = {"sentence": sample}

    # update results dictionary with predictions for every sentence
    # zip() yields tuples containing the elements from the same indices
    # in the iterables passed as parameters until the shortest iterable
    # is exhausted.
    data_dict.update(
        {
            category: prob
            for category, prob in zip(go_df.aspect.cat.categories, prediction_list)
        }
    )

    data_list.append(data_dict)

# create dataframe with predictions
probs_df = pd.DataFrame(data_list)

# calculate mean for each aspect in the text
probs_df.loc[len(sentences_list)] = probs_df.mean(numeric_only=True)


probs_df = probs_df.style.format(
    {
        "biological_process": "{:.2%}",
        "cellular_component": "{:.2%}",
        "molecular_function": "{:.2%}",
    }
)

probs_df

## What Are You Paying Attention To? Find Out with BERT Visualizer

Some days, you stumble upon beautiful and useful tools. Today, that gem is [BertViz](https://github.com/jessevig/bertviz), an interactive tool for visualizing attention in transformer models.

In [ ]:
from bertviz import model_view, head_view, neuron_view

# sample text
sample = "Life is a bunch of chemical reactions that started about 4.5 billion years ago"

# tokenize inputs
inputs = tokenizer.encode(sample, padding=True,
                          truncation=True, return_tensors="pt")

# evaluate and return attentions and corresponding tokens
outputs = model.cpu()(inputs, output_attentions=True)
attention = outputs[-1]
tokens = tokenizer.convert_ids_to_tokens(inputs[0])

head_view(attention, tokens)
